# Starting the Environment
1. Open an Anaconda Prompt terminal,
   - On windows, Search `anaconda prompt`, click to open
2. Change directory (`cd`) into class directory on your computer,
   - `cd C:\JW\Clarkson\AE470`
3. Change directory into the repository on your computer,
   - `cd AE470_Sp25`
4. Fetch the latest class repository from GitHub,
   - **This will overwrite any changes you have made to files in your local repository directory, `AE470_Sp2025`.**
   - **Be sure to rename any files where you make changes that you want to keep.**
   - `git fetch origin`
5. Reset your local branch repository to match the remote branch,
   - `git reset --hard origin/main`
6. Activate the virtual python environment,
   - `conda activate ae470sp25`
7. Start a Jupyter notebook session in a browser window.  Type the following into an Anaconda Prompt window,
   - `jupyter notebook`
8. Using the Jupyter browser, open this notebook: `02_ae470_eom_numerical_integration.ipynb`.


In [ ]:
# Configure Jupyter so figures appear in the notebook
# %matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# Equations of Motion and Numerical Integration

## Equations of Motion

Consider two point masses, $ \mathnormal{m_1} $ and $ \mathnormal{m_2} $
  
![two-body problem inertial frame](images/tbp_inertial_pointmasses.png)

$ \mathcal{F}_I $ is an inertial frame.  An inertial frame is a coordinate frame where Newton's Laws apply.

$ \vec{\mathbf{r}}_1 $ is the position vector of mass $ \mathnormal{m_1} $. 

$ \vec{\mathbf{r}}_2 $ is the position vector of mass $ \mathnormal{m_2} $. 

$ \vec{\mathbf{r}}_{21} $ is the position vector of mass $ \mathnormal{m_2} $ with relative to $ \mathnormal{m_1} $.
\begin{equation}
\vec{\mathbf{r}}_{21} = \vec{\mathbf{r}}_{2} - \vec{\mathbf{r}}_{1}
\end{equation}


The force exerted by each mass can be determined by Newton's law of gravitation.

The force $ \mathnormal{m_2} $ exerts on $ \mathnormal{m_1} $:
\begin{equation}
\vec{\mathbf{F}}_{12} = \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

The force $ \mathnormal{m_1} $ exerts on $ \mathnormal{m_2} $:
\begin{equation}
\vec{\mathbf{F}}_{21} = - \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

$\mathnormal{G}$ is Newton's universal gravitational constant.  
\begin{equation}
\mathnormal{G} = 6.67 \times 10^{-11}  \mathrm{m}^3 / (\mathrm{kg} \cdot \mathrm{s}^2)
\end{equation}

Applying Newton's second law:

$ \mathnormal{m_1} $:  
\begin{equation}
\mathnormal{m_1} \ddot{\vec{\mathbf{r}}}_{1} = \vec{\mathbf{F}}_{12} = \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

$ \mathnormal{m_2} $:
\begin{equation}
\mathnormal{m_2} \ddot{\vec{\mathbf{r}}}_{2} = \vec{\mathbf{F}}_{21} = - \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

Yields:

\begin{equation}
\ddot{\vec{\mathbf{r}}}_{1} = \frac{\mathnormal{G}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}
\begin{equation}
\ddot{\vec{\mathbf{r}}}_{2} = -\frac{\mathnormal{G}\mathnormal{m_1}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

Taking the difference:
\begin{equation}
\ddot{\vec{\mathbf{r}}}_{21} = \ddot{\vec{\mathbf{r}}}_{2} - \ddot{\vec{\mathbf{r}}}_{1} = - \frac{\mathnormal{G}(\mathnormal{m_1}+\mathnormal{m_2})}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

If $ \mathnormal{m_2} \ll \mathnormal{m_1}$, such as a satellite orbiting the Earth, we can assume $ \mathnormal{m_1} + \mathnormal{m_2} \approx \mathnormal{m_1} $ and define $ \mu = \mathnormal{G}\mathnormal{m_1} $, where $ \mathnormal{m_1} $ is the primary body and $ \mathnormal{m_2} $ is the secondary body.

After dropping the subscripts, the relative equations of motion are:
\begin{equation}
\boxed{ \ddot{\vec{\mathbf{r}}} = - \frac{\mu}{|\vec{\mathbf{r}}|^3} \vec{\mathbf{r}} }
\end{equation}

This is a vector nonlinear second-order differential equation that is only a function of $ \mathnormal{m_2} $'s position.

### Standard Gravitational Parameter

Earth, $ \oplus $:  
\begin{equation}
\mu_{\oplus} = 3.986 \times 10^{14}  \mathrm{m}^3 / \mathrm{s}^2 
\end{equation}

Sun, $ \odot $:  
\begin{equation}
\mu_{\odot} = 1.327 \times 10^{20}  \mathrm{m}^3 / \mathrm{s}^2 
\end{equation}


In [ ]:
try:
    from modsim import *
except ImportError:
    print("Download modsim from the course repository at https://github.com/jeffwalton/AE470_Sp25")

In [ ]:
import numpy as np
from matplotlib.pyplot import plot

### How do we solve differential equations numerically?

#### Simple example of projectile motion

A lacrosse ball is thrown at an angle $ \theta $ to the horizontal at a speed of 40.23 m/s.  Determine the path of motion, trajectory, of the ball if $ \theta $ = 38.0 deg.

\begin{equation}
\vec{\mathbf{a}} = \ddot{\vec{\mathbf{r}}} = 0\hat{\mathbf{i}} -9.8 \mathrm{m} / \mathrm{s}^2 \hat{\mathbf{j}}
\end{equation}

This is a second order linear ordinary differential equation.  To use the numerical integration module, it has to be written as a system of first-order differential equations.

First, define the state array for this problem as:
\begin{equation}
state = \begin{bmatrix}
          x \\
          y \\
          v_x \\
          v_y
        \end{bmatrix}
\end{equation}

We will use the the `State` object type from the `modsim` module to hold the state of the ball.


In [ ]:
v_mag = 40.23   # m/s
theta_deg = 38.0
theta = 38.0 * np.pi / 180  # convert degrees to radians, alternatively np.deg2rad(theta)
init = State(x=0, y=2.0, vx=v_mag * np.cos(theta), vy=v_mag * np.sin(theta))  # initial conditions

A `System` object will hold the ball's initial state, `init`, and other simulation parameters, for this example the acceleration due to gravity, `g`, and the end time of the simulation, `t_end`.

In [ ]:
system = System(init=init,
                g=9.8,
                t_end=10)

`init` and `system` are container variables holding the initial conditions and system parameters.

Create a 'slope' function containing the equations of motion for a simple projectile.  We write the second-order vector differential equation as a system of first-order scalar differential equations. 

\begin{gather}
\dot{x} = \frac{\mathrm d x}{\mathrm d t} = v_x \\
\dot{y} = \frac{\mathrm d y}{\mathrm d t} = v_y \\
\ddot{x} = \frac{\mathrm d v_x}{\mathrm d t} = 0.0 \\
\ddot{y} = \frac{\mathrm d v_y}{\mathrm d t} = -g
\end{gather}


In [ ]:
def eom_projectile(t, state, system):
    x, y, vx, vy = state  # unpack the individual variables from the state variable

    dxdt = vx
    dydt = vy
    d2xdt = 0.0
    d2ydt = -system.g

    return dxdt, dydt, d2xdt, d2ydt 
    

In [ ]:
# test the eom with the initial conditions
dxdt, dydt, d2xdt, d2ydt = eom_projectile(0.0, system.init, system)
dxdt, dydt, d2xdt, d2ydt

We will use a wrapper function from `modsim` to propagate the equations from $ t = 0.0 $ to $ t = t_{end}$.  `run_solve_ivp()` simplifies some of the parameters of the SciPy function [scipy.integrate.solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html), SciPy's initial value problem solver.  It also handles some overhead, like storing the results in a pandas DataFrame.  By default, `run_solve_ivp()` uses a RK45 method, a Runge-Kutta method of order 5(4) where error is controlled by comparing fourth- and fifth-order solutions.

`run_solve_ivp()` returns two variables, `results` (the DataFrame of the state variables at each time step) and `details` (information about the simulation run).

In [ ]:
results, details = run_solve_ivp(system, eom_projectile)
details.message

We can look at `results` and `details`.  


In [ ]:
results

The index (the value in the left column) of the DataFrame is time, $t$.

`details` contains information about the execution of `run_solve_ivp()` 

In [ ]:
details

We can make simple plots directly from the results DataFrame.

Plot y vs. time

In [ ]:
results.y.plot()

decorate(xlabel='Time (s)', ylabel='Height (m)')

Plot y vs. x

In [ ]:
plot(results.x, results.y, label=f'{theta_deg} degrees', color='C2')

decorate(xlabel='range (m)', ylabel='height (m)', title='Lacrosse Ball Projectile')

Whoa! It looks like the ball went below the surface of the ground!

`modsim` provides access to the `events` in `scipy.integrate.solve_ivp()` that allows us to stop the simulation when an event has a value of zero.  Let's define an event for the the ground surface, $y=0$ and re-run the simulation.

In [ ]:
def event_func(t, state, system):
    x, y, vx, vy = state

    return y

In [ ]:
results, details = run_solve_ivp(system, eom_projectile, events=event_func)
details.message

In [ ]:
plot(results.x, results.y, label=f'{theta_deg} degrees', color='C2')

decorate(xlabel='range (m)', ylabel='height (m)', title='Lacrosse Ball Projectile')

`head()` and `tail()` functions return the first and last rows of the DataFrame, respectively.

In [ ]:
results.head()

In [ ]:
results.tail(2)

* At what time did did the ball hit the ground?

The pandas DataFrame [iloc(index) property](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html) returns data at the `index` integer position. Remember Python has zero-based array indicies.

In [ ]:
results.iloc[2]

The `[-1]` position returns the last row.

In [ ]:
results.iloc[-1]

Since index is the time value, `.index[-1]` will give us the time of the last row -- when the ball hit the ground.  

In [ ]:
results.index[-1]

* How far did the ball go horizontally?

Let's add some [format specifications](https://docs.python.org/3/library/string.html#formatspec).

In [ ]:
print(f"The ball flew {results.iloc[-1].x} meters horizonally.")

In [ ]:
print(f"The ball flew {results.iloc[-1].x:.1f} meters horizonally in {results.index[-1]:.2f} seconds.")

### Now back to the two-body orbit problem...

#### Example Two-Body Orbit

A 1000 kg satellite with initial position vector $ \vec{\mathbf{r}} = 8000\hat{\mathbf{i}} + 6000\hat{\mathbf{k}} (\mathrm{km}) $ and initial velocity $ \vec{\mathbf{v}} = 7\hat{\mathbf{j}} (\mathrm{km/s}) $ is orbiting the earth.  Solve for the path of the satellite over the next 4 hours. (Ref: Curtis, Example 2.3)


Let's define an initial state vector and a system object for the simulation.

In [ ]:
r0 = Vector( 8000, 0, 6000 ) # km
v0 = Vector( 0, 7, 0 ) # km/s

init = State(x=r0.x, y=r0.y, z=r0.z, vx=v0.x, vy=v0.y, vz=v0.z)

system = System(init=init,
                mu=3.986000e+5, # earth, km^3/s^2
                t_end=4 * 60 * 60, # s
               )

show(system)

Now, the equations of motion for the two-body problem written as a system of first-order differential equations.

In [ ]:
def eom_twobodies(t, state, system):
    x, y, z, vx, vy, vz = state  # unpack the state vector
    mu = system.mu
    
    R = Vector(x, y, z)
    r = vector_mag(R)
    V = Vector(vx, vy, vz)
    # print(f"time:{t}, R:[{R.x}, {R.y}, {R.z}]")

    R_dot = V
    R_dotdot = -mu / r**3 * R

    return R_dot.x, R_dot.y, R_dot.z, R_dotdot.x, R_dotdot.y, R_dotdot.z
    
    

Test the equations of motion with the initial conditions.

In [ ]:
eom_twobodies(0, init, system)

Run the simulation for a short time interval, say 5 minutes.  The [SciPy solve_ivp()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) optional parameter `first_step` has been added to specify the the initial step size.

In [ ]:
system = System(init=init,
                mu=3.986000e+5, # km^3/s^2
                t_end=5 * 60, # s
               )

results, details = run_solve_ivp(system, eom_twobodies, first_step=10.0, rtol=1.0e-6)

details.message

In [ ]:
results.head()

## Exercises

1. Uncomment the print statement in the EOM function.  Run the simulation. What do you observe?
2. Uncomment the print statement in the EOM function and remove the `first_step` parameter.  Run the simulation.  What is happening here?


#### Run 2-body example for entire 4 hours

Return the comment to the print statement in the EOM function.

Now run for the full interval, 4 hours.

In [ ]:
system = System(init=init,
                mu=3.986000e+5, # km^3/s^2
                t_end=4 * 60 * 60, # s
               )

results, details = run_solve_ivp(system, eom_twobodies, first_step=10.0, rtol=1.0e-6)

details.message

In [ ]:
results.tail()

#### Plot the position in 3 dimensions

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(results.x, results.y, results.z)
ax.set_title("First Orbit 3D plot")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.scatter(0, 0, 0)


plt.show()

#### Can we get interactive plots to work?

The [plotly](https://plotly.com/python/) library allows simple, interactive 3D plotting.

plotly can be installed by typing `conda install plotly` in your `ae470sp25` virtual environment.
1. Shutdown this notebook.
2. In the Anaconda Prompt window, run `conda install plotly`.
3. Restart Jupyter, `jupyter notebook`
4. Reopen this notebook and run cells to this point.

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
fig = go.Figure(
    data=[go.Scatter3d(
        x=results.x,
        y=results.y,
        z=results.z,
        mode='markers',
        marker=dict(
            size=2
        )
    )
         ]
    )
fig.show()

We can add the magnitude of the velocity to the results DataFrame.

In [ ]:
def vector_mag_df(v_x, v_y, v_z):
    return vector_mag(Vector(v_x, v_y, v_z))


In [ ]:
results['v_mag'] = results.apply(lambda row: vector_mag_df(row.vx, row.vy, row.vz), axis=1)

In [ ]:
results.head()

In [ ]:
fig = px.scatter_3d(results, x='x', y='y', z='z',
              color='v_mag', 
              )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()